# Preprocessing

In [ ]:
!pip install nltk
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.model_selection import GridSearchCV


In [ ]:
df_read = pd.read_csv("/content/drive/MyDrive/CIS_450_Final/Final Datasets/descriptions.csv")

In [ ]:

sample=df_read
sample = sample.drop_duplicates()

sample['content']=sample['summary'].astype(str)+' ' + sample['description'].astype(str)+"  " + sample['neighborhood_overview'].astype(str)+"  " + sample['notes'].astype(str)
sample

,Unnamed: 0,listing_id,name,summary,space,description,neighborhood_overview,notes,transit,content
0,0,241032,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,NaN,NaN,NaN,nan Make your self at home in this charming on...
1,1,953595,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,"Queen Anne is a wonderful, truly functional vi...",What's up with the free pillows? Our home was...,"Convenient bus stops are just down the block, ...",Chemically sensitive? We've removed the irrita...
2,2,3308979,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,Upper Queen Anne is a charming neighborhood fu...,Our house is located just 5 short blocks to To...,A bus stop is just 2 blocks away. Easy bus a...,New modern house built in 2013. Spectacular s...
3,3,7421966,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,NaN,NaN,NaN,A charming apartment that sits atop Queen Anne...
4,4,278830,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,We are in the beautiful neighborhood of Queen ...,Belltown,The nearest public transit bus (D Line) is 2 b...,Cozy family craftman house in beautiful neighb...
...,...,...,...,...,...,...,...,...,...,...
3813,3813,8101950,3BR Mountain View House in Seattle,Our 3BR/2BA house boasts incredible views of t...,"Our 3BR/2BA house bright, stylish, and wheelch...",Our 3BR/2BA house boasts incredible views of t...,We're located near lots of family fun. Woodlan...,NaN,NaN,Our 3BR/2BA house boasts incredible views of t...
3814,3814,8902327,Portage Bay View!-One Bedroom Apt,800 square foot 1 bedroom basement apartment w...,This space has a great view of Portage Bay wit...,800 square foot 1 bedroom basement apartment w...,The neighborhood is a quiet oasis that is clos...,This is a basement apartment in a newer reside...,Uber and Car2go are good options in Seattle. T...,800 square foot 1 bedroom basement apartment w...
3815,3815,10267360,Private apartment view of Lake WA,"Very comfortable lower unit. Quiet, charming m...",NaN,"Very comfortable lower unit. Quiet, charming m...",NaN,NaN,NaN,"Very comfortable lower unit. Quiet, charming m..."
3816,3816,9604740,Amazing View with Modern Comfort!,Cozy studio condo in the heart on Madison Park...,Fully furnished unit to accommodate most needs...,Cozy studio condo in the heart on Madison Park...,Madison Park offers a peaceful slow pace upsca...,NaN,Yes,Cozy studio condo in the heart on Madison Park...


Start preprocessing the textual data

In [ ]:
sample_data = sample
import re
sample_data['content_p'] = sample_data['content'].astype(str)
#lower case
sample_data['content_p'] = sample_data['content_p'].map(lambda x: x.lower())
#stopword removal
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
pat = r'\b(?:{})\b'.format('|'.join(stop))
#remove punctuation
sample_data['content_p'] = sample_data['content_p'].str.replace(pat, '')
sample_data['content_p'] = sample_data['content_p'].str.replace(r'\s+', ' ')
sample_data['content_p'] = sample_data['content_p'].map(lambda x: re.sub('[,\.!?’‘]', '', x))
#lemmatize
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer=WordNetLemmatizer()
def subLemma(word):
  return lemmatizer.lemmatize(word) #lemmatize
  #find_result = list(map(word.find, remove_list))

  #if (sum(find_result) > -1 * len(find_result)):
    #return ''
  #else:
    #return word  
def lemma (input_str):
  input_str=word_tokenize(input_str)
  input_str = list(map(subLemma, input_str))
  return ' '.join(input_str)
  

sample_data['content_p'] = sample_data['content_p'].apply(lambda x: lemma(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
#remove numbers
sample_data['content_p'] = sample_data['content_p'].apply(lambda x: re.sub(r'\d+', '', x))

In [ ]:
sample_data['processed_content']=sample_data['content_p']
df_train = sample_data
df_test = df_train

# LDA

In [ ]:
#Bag of Words

from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Initialise the count vectorizer with the English stop words
count_vectorizer = CountVectorizer(stop_words='english', max_df=0.1, min_df=0.001)
# Fit and transform the processed titles
count_data_train = count_vectorizer.fit_transform(df_train['processed_content'])


In [ ]:
texts = df_train['processed_content']
texts.append(df_test['processed_content'])

0       nan make self home charming one-bedroom apartm...
1       chemically sensitive ' removed irritant trigge...
2       new modern house built  spectacular sunset/wat...
3       charming apartment sits atop queen anne hill n...
4       cozy family craftman house beautiful neighborh...
                              ...                        
3813    br/ba house boast incredible view olympic moun...
3814     square foot  bedroom basement apartment fanta...
3815    comfortable lower unit quiet charming mid-cent...
3816    cozy studio condo heart madison park lake wa s...
3817    hardwood floor fireplace  '' tv xbox one eleva...
Name: processed_content, Length: 7636, dtype: object

In [ ]:
count_v = count_vectorizer.fit(texts)

In [ ]:
count_test = count_v.transform(df_test['processed_content'])
count_train = count_v.transform(df_train['processed_content'])

In [ ]:
count_data_test = count_vectorizer.fit_transform(df_train['processed_content'])

In [ ]:
train_titles= df_train['summary'].tolist()
test_titles =df_test['summary'].tolist()

Seeing the LDA on topics 20 and how its topics look like

In [ ]:
lda_20 = LDA(n_components=20, n_jobs=-1, random_state=42)
lda_20.fit(count_train)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=20, n_jobs=-1,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [ ]:
!pip install pyldavis

import pyLDAvis
import pyLDAvis.sklearn

data = pyLDAvis.sklearn.prepare(lda_20, count_train, count_vectorizer)
pyLDAvis.display(data)

In [ ]:
W_test = lda_20.transform(count_train)
df_train['prob_topic_1']=W_test[:,:1]
df_train['prob_topic_2']=W_test[:,1:2]
df_train['prob_topic_3']=W_test[:,2:3]
df_train['prob_topic_4']=W_test[:,3:4]
df_train['prob_topic_5']=W_test[:,4:5]
df_train['prob_topic_6']=W_test[:,5:6]
df_train['prob_topic_7']=W_test[:,6:7]
df_train['prob_topic_8']=W_test[:,7:8]
df_train['prob_topic_9']=W_test[:,8:9]
df_train['prob_topic_10']=W_test[:,9:10]
df_train['prob_topic_11']=W_test[:,10:11]
df_train['prob_topic_12']=W_test[:,11:12]
df_train['prob_topic_13']=W_test[:,12:13]
df_train['prob_topic_14']=W_test[:,13:14]
df_train['prob_topic_15']=W_test[:,14:15]
df_train['prob_topic_16']=W_test[:,15:16]
df_train['prob_topic_17']=W_test[:,16:17]
df_train['prob_topic_18']=W_test[:,17:18]
df_train['prob_topic_19']=W_test[:,18:19]
df_train['prob_topic_20']=W_test[:,19:20]
df_train

,Unnamed: 0,listing_id,name,summary,space,description,neighborhood_overview,notes,transit,content,content_p,processed_content,prob_topic_1,prob_topic_2,prob_topic_3,prob_topic_4,prob_topic_5,prob_topic_6,prob_topic_7,prob_topic_8,prob_topic_9,prob_topic_10,prob_topic_11,prob_topic_12,prob_topic_13,prob_topic_14,prob_topic_15,prob_topic_16,prob_topic_17,prob_topic_18,prob_topic_19,prob_topic_20
0,0,241032,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,NaN,NaN,NaN,nan Make your self at home in this charming on...,nan make self home charming one-bedroom apartm...,nan make self home charming one-bedroom apartm...,0.001613,0.114708,0.001613,0.001613,0.001613,0.001613,0.152182,0.001613,0.001613,0.001613,0.001613,0.001613,0.001613,0.222947,0.001613,0.001613,0.001613,0.001613,0.484356,0.001613
1,1,953595,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,"Queen Anne is a wonderful, truly functional vi...",What's up with the free pillows? Our home was...,"Convenient bus stops are just down the block, ...",Chemically sensitive? We've removed the irrita...,chemically sensitive ' removed irritant trigge...,chemically sensitive ' removed irritant trigge...,0.000333,0.993667,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333
2,2,3308979,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,Upper Queen Anne is a charming neighborhood fu...,Our house is located just 5 short blocks to To...,A bus stop is just 2 blocks away. Easy bus a...,New modern house built in 2013. Spectacular s...,new modern house built spectacular sunset/wat...,new modern house built spectacular sunset/wat...,0.000485,0.233607,0.060120,0.000485,0.000485,0.000485,0.698020,0.000485,0.000485,0.000485,0.000485,0.000485,0.000485,0.000485,0.000485,0.000485,0.000485,0.000485,0.000485,0.000485
3,3,7421966,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,NaN,NaN,NaN,A charming apartment that sits atop Queen Anne...,charming apartment sits atop queen anne hill n...,charming apartment sits atop queen anne hill n...,0.001429,0.089243,0.001429,0.001429,0.001429,0.001429,0.642144,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.244327,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429
4,4,278830,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,We are in the beautiful neighborhood of Queen ...,Belltown,The nearest public transit bus (D Line) is 2 b...,Cozy family craftman house in beautiful neighb...,cozy family craftman house beautiful neighborh...,cozy family craftman house beautiful neighborh...,0.000877,0.597022,0.000877,0.000877,0.000877,0.000877,0.239561,0.000877,0.000877,0.000877,0.000877,0.000877,0.000877,0.148505,0.000877,0.000877,0.000877,0.000877,0.000877,0.000877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,3813,8101950,3BR Mountain View House in Seattle,Our 3BR/2BA house boasts incredible views of t...,"Our 3BR/2BA house bright, stylish, and wheelch...",Our 3BR/2BA house boasts incredible views of t...,We're located near lots of family fun. Woodlan...,NaN,NaN,Our 3BR/2BA house boasts incredible views of t...,br/ba house boast incredible view olympic moun...,br/ba house boast incredible view olympic moun...,0.000588,0.000588,0.000588,0.112299,0.000588,0.000588,0.430364,0.240066,0.000588,0.000588,0.000588,0.207859,0.

In [ ]:
result=df_train.drop(columns=['name','summary','space','description','neighborhood_overview','notes','transit','content','content_p','processed_content'])
result

,Unnamed: 0,listing_id,prob_topic_1,prob_topic_2,prob_topic_3,prob_topic_4,prob_topic_5,prob_topic_6,prob_topic_7,prob_topic_8,prob_topic_9,prob_topic_10,prob_topic_11,prob_topic_12,prob_topic_13,prob_topic_14,prob_topic_15,prob_topic_16,prob_topic_17,prob_topic_18,prob_topic_19,prob_topic_20
0,0,241032,0.001613,0.114708,0.001613,0.001613,0.001613,0.001613,0.152182,0.001613,0.001613,0.001613,0.001613,0.001613,0.001613,0.222947,0.001613,0.001613,0.001613,0.001613,0.484356,0.001613
1,1,953595,0.000333,0.993667,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333
2,2,3308979,0.000485,0.233607,0.060120,0.000485,0.000485,0.000485,0.698020,0.000485,0.000485,0.000485,0.000485,0.000485,0.000485,0.000485,0.000485,0.000485,0.000485,0.000485,0.000485,0.000485
3,3,7421966,0.001429,0.089243,0.001429,0.001429,0.001429,0.001429,0.642144,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429,0.244327,0.001429,0.001429,0.001429,0.001429,0.001429,0.001429
4,4,278830,0.000877,0.597022,0.000877,0.000877,0.000877,0.000877,0.239561,0.000877,0.000877,0.000877,0.000877,0.000877,0.000877,0.148505,0.000877,0.000877,0.000877,0.000877,0.000877,0.000877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,3813,8101950,0.000588,0.000588,0.000588,0.112299,0.000588,0.000588,0.430364,0.240066,0.000588,0.000588,0.000588,0.207859,0.000588,0.000588,0.000588,0.000588,0.000588,0.000588,0.000588,0.000588
3814,3814,8902327,0.000926,0.000926,0.000926,0.000926,0.000926,0.000926,0.202675,0.000926,0.017393,0.000926,0.000926,0.000926,0.000926,0.241420,0.272029,0.122292,0.000926,0.000926,0.000926,0.131228
3815,3815,10267360,0.003846,0.232435,0.003846,0.003846,0.003846,0.003846,0.003846,0.003846,0.003846,0.003846,0.003846,0.003846,0.003846,0.003846,0.003846,0.003846,0.003846,0.003846,0.003846,0.698334
3816,3816,9604740,0.000625,0.238429,0.000625,0.000625,0.046332,0.000625,0.000625,0.000625,0.000625,0.000625,0.042928,0.000625,0.425360,0.000625,0.000625,0.000625,0.000625,0.000625,0.237576,0.000625


In [ ]:
#result.to_csv("MLTopics.csv")
!cp MLTopics.csv "/content/drive/MyDrive/CIS_450_Final/"